In [1]:
from clustering import *
from downloaddata import *
from portfolio_and_performance import *
from trees import *
import cPickle as pickle

In [2]:
window = 100
space = 10
thresh = 95
filename = "SP500_from_1985.csv"
enddate='2017-06-28'
startdate='1985-01-02'

In [3]:
price = importdata("data/WRDS/SP500_price_1985.csv").replace(0,np.nan).ffill()
ret = importdata("data/WRDS/SP500_ret_1985.csv").fillna(0)
div = importdata("data/WRDS/SP500_div_1985.csv").fillna(0)
cpr = importdata("data/WRDS/SP500_cpr_1985.csv").replace(0,np.nan).ffill()

In [4]:
D = corr_matrix(ret, thresh, window=250, enddate="2016-12-30", method="gower", shrinkage="None")

In [5]:
S = np.exp(-D)

In [12]:
import matlab.engine
eng = matlab.engine.start_matlab()

In [18]:
S

matlab.double([[1.0,0.337877219035,0.329468599309,0.251725224214,0.282823485386,0.347590725689,0.388394876368,0.349202292561,0.382144645724,0.432004875611,0.410801470329,0.35787060676,0.333110727918,0.278808834032,0.335924952991,0.29577583216,0.32537444474,0.332738551842,0.334463820714,0.351001721519,0.350312808382,0.335722841929,0.349040863533,0.31408444215,0.365898790679,0.299701653789,0.362657900119,0.260712451219,0.336353204437,0.291844043061,0.27608633384,0.313811194525,0.328568236001,0.381345665187,0.328057252299,0.371127362596,0.333992278812,0.265516735469,0.380282324016,0.368928431817,0.351741744338,0.315467816126,0.297487814432,0.322221944045,0.371442703618,0.360916545664,0.327672542911,0.347401536295,0.314753337838,0.357200884889,0.245723206205,0.315330010325,0.358348008962,0.295505126833,0.331938766251,0.317260904578,0.391579851335,0.337609509257,0.357972618326,0.451330078525,0.318699628467,0.40206052409,0.31862380705,0.315911374015,0.292120307592,0.284339835171,0.2733887240

In [16]:
D = matlab.double(D.as_matrix().tolist())
S = matlab.double(S.as_matrix().tolist())

In [6]:
import numpy as np
import matlab.engine
eng = matlab.engine.start_matlab()
#eng.cd(r'C:\Users\RockLIANG\Desktop') #set up matlab wdir to where the matlab scripts are located

MatlabExecutionError: 
  File /home/xuxiao2695/PycharmProjects/clustering/MATLAB/pmfg.m, line 86, in pmfg
Undefined function 'boyer_myrvold_planarity_test' for input arguments of type 'logical'.


In [8]:
eng.cd(r'MATLAB')

'/home/xuxiao2695/PycharmProjects/clustering'

In [44]:
def DBHT(thresh, filename="SP100_prices.csv", window=250, enddate="2017-01-24", startdate='2015-12-30', space=1,
         shrinkage="None"):
    """Returns a dictionary of Minimum Spanning Tree for each end date,
    space means the interval between two sample updates"""
    ret = importdata(filename)
    dic = rolling_corr(ret, thresh, window, enddate, startdate, space, shrinkage=shrinkage)
    clusterings = {}
    PMFG = {}
    bubble_cluster_membership = {}
    PMFG_shortest_path_length_matrix = {}
    bubble_membership_mattrix = {}
    DBHT_hierarchy = {}
    eng = matlab.engine.start_matlab()
    eng.cd(r'MATLAB')
    count = 0
    for key in sorted(dic.keys()):
        count+=1
        D = dic[key]
        S = np.exp(-D)
        D = matlab.double(D.as_matrix().tolist())
        S = matlab.double(S.as_matrix().tolist())
        T8, Rpm, Adjv, Dpm, Mv, Z = eng.DBHTs(D, S, nargout=6)
        clusterings[key] = T8
        PMFG[key] = Rpm
        bubble_cluster_membership[key] = Adjv
        PMFG_shortest_path_length_matrix[key] = Dpm
        bubble_membership_mattrix[key] = Mv
        DBHT_hierarchy[key] = Z
        print "building DBHT for the number "+ str(count) + " date " + key + " done."
    return {'DBHT_clusterings': clusterings, 'PMFG': PMFG,
            'bubble_cluster_membership_matrix': bubble_membership_mattrix,
            'PMFG_shortest_paath_length_matrix': PMFG_shortest_path_length_matrix,
            'bubble_membership_matrix': bubble_membership_mattrix,
            'DBHT_hierarchy': DBHT_hierarchy}

In [36]:
result = eng.DBHTs(D,S)

In [46]:
DBHT_results = DBHT(thresh=95, filename = "data/WRDS/SP500_ret_1985.csv", window=100,
     enddate='2016-12-30', startdate='1985-01-02',space=10, shrinkage='None')

corr_matrix for 2016-12-30 done.
corr_matrix for 2016-12-15 done.
corr_matrix for 2016-12-01 done.
corr_matrix for 2016-11-16 done.
corr_matrix for 2016-11-02 done.
corr_matrix for 2016-10-19 done.
corr_matrix for 2016-10-05 done.
corr_matrix for 2016-09-21 done.
corr_matrix for 2016-09-07 done.
corr_matrix for 2016-08-23 done.
corr_matrix for 2016-08-09 done.
corr_matrix for 2016-07-26 done.
corr_matrix for 2016-07-12 done.
corr_matrix for 2016-06-27 done.
corr_matrix for 2016-06-13 done.
corr_matrix for 2016-05-27 done.
corr_matrix for 2016-05-13 done.
corr_matrix for 2016-04-29 done.
corr_matrix for 2016-04-15 done.


corr_matrix for 2016-04-01 done.
corr_matrix for 2016-03-17 done.
corr_matrix for 2016-03-03 done.
corr_matrix for 2016-02-18 done.
corr_matrix for 2016-02-03 done.
corr_matrix for 2016-01-20 done.
corr_matrix for 2016-01-05 done.
corr_matrix for 2015-12-18 done.
corr_matrix for 2015-12-04 done.
corr_matrix for 2015-11-19 done.
corr_matrix for 2015-11-05 done.
corr_matrix for 2015-10-22 done.
corr_matrix for 2015-10-08 done.
corr_matrix for 2015-09-24 done.
corr_matrix for 2015-09-10 done.
corr_matrix for 2015-08-26 done.
corr_matrix for 2015-08-12 done.
corr_matrix for 2015-07-29 done.
corr_matrix for 2015-07-15 done.


corr_matrix for 2015-06-30 done.
corr_matrix for 2015-06-16 done.
corr_matrix for 2015-06-02 done.
corr_matrix for 2015-05-18 done.
corr_matrix for 2015-05-04 done.
corr_matrix for 2015-04-20 done.
corr_matrix for 2015-04-06 done.
corr_matrix for 2015-03-20 done.
corr_matrix for 2015-03-06 done.
corr_matrix for 2015-02-20 done.
corr_matrix for 2015-02-05 done.
corr_matrix for 2015-01-22 done.
corr_matrix for 2015-01-07 done.
corr_matrix for 2014-12-22 done.
corr_matrix for 2014-12-08 done.
corr_matrix for 2014-11-21 done.
corr_matrix for 2014-11-07 done.
corr_matrix for 2014-10-24 done.
corr_matrix for 2014-10-10 done.


corr_matrix for 2014-09-26 done.
corr_matrix for 2014-09-12 done.
corr_matrix for 2014-08-28 done.
corr_matrix for 2014-08-14 done.
corr_matrix for 2014-07-31 done.
corr_matrix for 2014-07-17 done.
corr_matrix for 2014-07-02 done.
corr_matrix for 2014-06-18 done.
corr_matrix for 2014-06-04 done.
corr_matrix for 2014-05-20 done.
corr_matrix for 2014-05-06 done.
corr_matrix for 2014-04-22 done.
corr_matrix for 2014-04-07 done.
corr_matrix for 2014-03-24 done.
corr_matrix for 2014-03-10 done.
corr_matrix for 2014-02-24 done.
corr_matrix for 2014-02-07 done.
corr_matrix for 2014-01-24 done.
corr_matrix for 2014-01-09 done.


corr_matrix for 2013-12-24 done.
corr_matrix for 2013-12-10 done.
corr_matrix for 2013-11-25 done.
corr_matrix for 2013-11-11 done.
corr_matrix for 2013-10-28 done.
corr_matrix for 2013-10-14 done.
corr_matrix for 2013-09-30 done.
corr_matrix for 2013-09-16 done.
corr_matrix for 2013-08-30 done.
corr_matrix for 2013-08-16 done.
corr_matrix for 2013-08-02 done.
corr_matrix for 2013-07-19 done.
corr_matrix for 2013-07-05 done.
corr_matrix for 2013-06-20 done.
corr_matrix for 2013-06-06 done.
corr_matrix for 2013-05-22 done.
corr_matrix for 2013-05-08 done.
corr_matrix for 2013-04-24 done.


...

building DBHT for the number 803 date 2016-11-02 done.


building DBHT for the number 804 date 2016-11-16 done.


building DBHT for the number 805 date 2016-12-01 done.


building DBHT for the number 806 date 2016-12-15 done.


building DBHT for the number 807 date 2016-12-30 done.


In [47]:
pickle.dump(DBHT_results, open("pickle/DBHT_unshrunk.p", 'wb'))

In [48]:
DBHT_results_LedoitWolf = DBHT(thresh=95, filename = "data/WRDS/SP500_ret_1985.csv", window=100,
     enddate='2016-12-30', startdate='1985-01-02',space=10, shrinkage='LedoitWolf')
pickle.dump(DBHT_results_LedoitWolf, open("pickle/DBHT_Ledoit_wolf.p", 'wb'))

corr_matrix for 2016-12-30 done.
corr_matrix for 2016-12-15 done.
corr_matrix for 2016-12-01 done.
corr_matrix for 2016-11-16 done.
corr_matrix for 2016-11-02 done.
corr_matrix for 2016-10-19 done.
corr_matrix for 2016-10-05 done.
corr_matrix for 2016-09-21 done.
corr_matrix for 2016-09-07 done.
corr_matrix for 2016-08-23 done.
corr_matrix for 2016-08-09 done.
corr_matrix for 2016-07-26 done.
corr_matrix for 2016-07-12 done.
corr_matrix for 2016-06-27 done.
corr_matrix for 2016-06-13 done.
corr_matrix for 2016-05-27 done.
corr_matrix for 2016-05-13 done.
corr_matrix for 2016-04-29 done.
corr_matrix for 2016-04-15 done.
corr_matrix for 2016-04-01 done.
corr_matrix for 2016-03-17 done.
corr_matrix for 2016-03-03 done.
corr_matrix for 2016-02-18 done.
corr_matrix for 2016-02-03 done.
corr_matrix for 2016-01-20 done.
corr_matrix for 2016-01-05 done.
corr_matrix for 2015-12-18 done.
corr_matrix for 2015-12-04 done.
corr_matrix for 2015-11-19 done.
corr_matrix for 2015-11-05 done.
corr_matri

corr_matrix for 2014-12-08 done.
corr_matrix for 2014-11-21 done.
corr_matrix for 2014-11-07 done.
corr_matrix for 2014-10-24 done.
corr_matrix for 2014-10-10 done.
corr_matrix for 2014-09-26 done.
corr_matrix for 2014-09-12 done.
corr_matrix for 2014-08-28 done.
corr_matrix for 2014-08-14 done.
corr_matrix for 2014-07-31 done.
corr_matrix for 2014-07-17 done.
corr_matrix for 2014-07-02 done.
corr_matrix for 2014-06-18 done.
corr_matrix for 2014-06-04 done.
corr_matrix for 2014-05-20 done.
corr_matrix for 2014-05-06 done.
corr_matrix for 2014-04-22 done.
corr_matrix for 2014-04-07 done.
corr_matrix for 2014-03-24 done.
corr_matrix for 2014-03-10 done.
corr_matrix for 2014-02-24 done.
corr_matrix for 2014-02-07 done.
corr_matrix for 2014-01-24 done.
corr_matrix for 2014-01-09 done.
corr_matrix for 2013-12-24 done.
corr_matrix for 2013-12-10 done.
corr_matrix for 2013-11-25 done.
corr_matrix for 2013-11-11 done.
corr_matrix for 2013-10-28 done.
corr_matrix for 2013-10-14 done.
corr_matri

corr_matrix for 2013-02-11 done.
corr_matrix for 2013-01-28 done.
corr_matrix for 2013-01-11 done.
corr_matrix for 2012-12-27 done.
corr_matrix for 2012-12-12 done.
corr_matrix for 2012-11-28 done.
corr_matrix for 2012-11-13 done.
corr_matrix for 2012-10-26 done.
corr_matrix for 2012-10-12 done.
corr_matrix for 2012-09-28 done.
corr_matrix for 2012-09-14 done.
corr_matrix for 2012-08-30 done.
corr_matrix for 2012-08-16 done.
corr_matrix for 2012-08-02 done.
corr_matrix for 2012-07-19 done.
corr_matrix for 2012-07-05 done.
corr_matrix for 2012-06-20 done.
corr_matrix for 2012-06-06 done.
corr_matrix for 2012-05-22 done.
corr_matrix for 2012-05-08 done.
corr_matrix for 2012-04-24 done.
corr_matrix for 2012-04-10 done.
corr_matrix for 2012-03-26 done.
corr_matrix for 2012-03-12 done.
corr_matrix for 2012-02-27 done.
corr_matrix for 2012-02-10 done.
corr_matrix for 2012-01-27 done.
corr_matrix for 2012-01-12 done.
corr_matrix for 2011-12-28 done.
corr_matrix for 2011-12-13 done.
corr_matri

corr_matrix for 2011-04-27 done.
corr_matrix for 2011-04-12 done.
corr_matrix for 2011-03-29 done.
corr_matrix for 2011-03-15 done.
corr_matrix for 2011-03-01 done.
corr_matrix for 2011-02-14 done.
corr_matrix for 2011-01-31 done.
corr_matrix for 2011-01-14 done.
corr_matrix for 2010-12-31 done.
corr_matrix for 2010-12-16 done.
corr_matrix for 2010-12-02 done.
corr_matrix for 2010-11-17 done.
corr_matrix for 2010-11-03 done.
corr_matrix for 2010-10-20 done.
corr_matrix for 2010-10-06 done.
corr_matrix for 2010-09-22 done.
corr_matrix for 2010-09-08 done.
corr_matrix for 2010-08-24 done.
corr_matrix for 2010-08-10 done.
corr_matrix for 2010-07-27 done.
corr_matrix for 2010-07-13 done.
corr_matrix for 2010-06-28 done.
corr_matrix for 2010-06-14 done.
corr_matrix for 2010-05-28 done.
corr_matrix for 2010-05-14 done.
corr_matrix for 2010-04-30 done.
corr_matrix for 2010-04-16 done.
corr_matrix for 2010-04-01 done.
corr_matrix for 2010-03-18 done.
corr_matrix for 2010-03-04 done.
corr_matri

corr_matrix for 2009-06-30 done.
corr_matrix for 2009-06-16 done.
corr_matrix for 2009-06-02 done.
corr_matrix for 2009-05-18 done.
corr_matrix for 2009-05-04 done.
corr_matrix for 2009-04-20 done.
corr_matrix for 2009-04-03 done.
corr_matrix for 2009-03-20 done.
corr_matrix for 2009-03-06 done.
corr_matrix for 2009-02-20 done.
corr_matrix for 2009-02-05 done.
corr_matrix for 2009-01-22 done.
corr_matrix for 2009-01-07 done.
corr_matrix for 2008-12-22 done.
corr_matrix for 2008-12-08 done.
corr_matrix for 2008-11-21 done.
corr_matrix for 2008-11-07 done.
corr_matrix for 2008-10-24 done.
corr_matrix for 2008-10-10 done.
corr_matrix for 2008-09-26 done.
corr_matrix for 2008-09-12 done.
corr_matrix for 2008-08-28 done.
corr_matrix for 2008-08-14 done.
corr_matrix for 2008-07-31 done.
corr_matrix for 2008-07-17 done.
corr_matrix for 2008-07-02 done.
corr_matrix for 2008-06-18 done.
corr_matrix for 2008-06-04 done.
corr_matrix for 2008-05-20 done.
corr_matrix for 2008-05-06 done.
corr_matri

...

building DBHT for the number 803 date 2016-11-02 done.


building DBHT for the number 804 date 2016-11-16 done.


building DBHT for the number 805 date 2016-12-01 done.


building DBHT for the number 806 date 2016-12-15 done.


building DBHT for the number 807 date 2016-12-30 done.


In [36]:
result = eng.DBHTs(D,S)

In [33]:
# pmfg funcion----------------------------------------
"""
P = pmfg(W)

pmfg calculate the the planar maximally filtered  graph (PMFG) from a matrix of weights W. 
Input:
W should be sparse, real, square and symmetric matrix
Output:
P = pmfg(W) returns a matrix with P(i,j)=W(i,j) if the edge i-j is present and P(i,j)=0 if not.

"""
#i.e.
P = eng.pmfg(D)
P = np.array(P) #change matlab double back to python np.array

MatlabExecutionError: 
  File /home/xuxiao2695/PycharmProjects/clustering/MATLAB/pmfg.m, line 86, in pmfg
Undefined function 'boyer_myrvold_planarity_test' for input arguments of type 'logical'.


In [46]:
# DBHT function---------------------------------------
"""
[T8,Rpm,Adjv,Dpm,Mv,Z]=DBHTs(D,S)

Perform DBHT clustering, a deterministic technique which only requires a
similarity matrix S, and related dissimilarity matrix D. 
Input:
 D = NxN dissimilarity matrix - e.g. a distance
 S = NxN similarity matrix (non-negative) - e.g. correlation coefficiant+1
Output:
 T8 = Nx1 cluster membership vector. 
 Rpm = NxN adjacency matrix of PMFG. 
 Adjv = Bubble cluster membership matrix from BubbleCluster8. 
 Adjv = N x Nk cluster membership matrix for vertices for non-discrete
 clustering via the bubble topology. Adjv(n,k)=1 indicates cluster

 membership of vertex n to kth non-discrete cluster.
 Dpm = NxN shortest path length matrix of PMFG
 Mv = NxNb bubble membership matrix. Nv(n,bi)=1 indicates vertex n is a
 vertex of bubble bi. 
 Z DBHT hierarchy
"""
#i.e.
#S = eng.corr(eng.randn(80,100))
#D = eng.corr(eng.randn(80,100))
T8,Rpm,Adjv,Dpm,Mv,Z= eng.DBHTs(D,S, nargout=6 ) #nargout determines the number of output

In [32]:
np.array(Adjv)

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.]])